In [14]:
import numpy as np
import pandas as pd
from phmd import datasets
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, RobustScaler
from scipy.stats import kurtosis
from scipy.fft import fft
from scipy.fft import fft, fftfreq
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
from tqdm import tqdm

In [3]:
def create_sequences(X, y, window_size=256, step=50):
    X_seq, y_seq = [], []

    for i in range(0, len(X) - window_size, step):
        X_seq.append(X[i:i+window_size])
        y_seq(y[i+window_size])

    return np.array(X_seq), np.array(y_seq)


def calculate_r2(preds, targets):
    # Convert tensors to numpy for sklearn
    preds = preds.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    return r2_score(targets, preds)

In [4]:
TRAIN_BEARINGS = {
    1: [
        '1_1',
        '1_2',
    ],
    2: [
        '2_1',
        '2_2',
    ],
    3: [
        '3_1',
        '3_2',
    ]
}

TEST_BEARINGS = {
    1: [
        '1_3',
        '1_4',
        '1_5',
        '1_6',
        '1_7',
    ],
    2: [
        '2_3',
        '2_4',
        '2_5',
        '2_6',
        '2_7',
    ],
    3: [
        '3_3',
    ]
}

dataset = datasets.Dataset('PRONOSTIA')
tasks = dataset['rul']
df = tasks.load()

Remember to cite the original publisher dataset:
	@inproceedings{nectoux2012pronostia,    
	    title={PRONOSTIA: An experimental platform for bearings accelerated degradation tests.},    
	    author={Nectoux, Patrick and Gouriveau, Rafael and Medjaher, Kamal and Ramasso, Emmanuel and Chebel-Morello, Brigitte and Zerhouni, Noureddine and Varnier, Christophe},    
	    booktitle={IEEE International Conference on Prognostics and Health Management, PHM'12.},    
	    pages={1--8},    
	    year={2012},    
	    organization={IEEE Catalog Number: CPF12PHM-CDR}    
	}
You can download the dataset manually from:  https://www.nasa.gov/intelligent-systems-division/discovery-and-systems-health/pcoe/pcoe-data-set-repository/

** If you find this tool useful, please cite our SoftwareX paper: 
	Solís-Martín, David, Juan Galán-Páez, and Joaquín Borrego-Díaz. "PHMD: An easy data access tool for prognosis and health management datasets." SoftwareX 29 (2025): 102039.



Reading Bearing3_3: 100%|██████████| 13959/13959 [00:37<00:00, 369.69it/s]
INFO:root:Read in 62.68148684501648 seconds


In [5]:
# train
df[0]['unit'].unique()

array(['1_1', '1_2', '2_1', '2_2', '3_1', '3_2'], dtype=object)

In [6]:
df[0].head(5)

,H_acc,rul,unit,V_acc
0,0.552,28029,1_1,-0.146
1,0.501,28029,1_1,-0.480
2,0.138,28029,1_1,0.435
3,-0.423,28029,1_1,0.240
4,-0.802,28029,1_1,0.020


In [7]:
# test
df[1]['unit'].unique()

array(['1_3', '1_4', '1_5', '1_6', '1_7', '2_3', '2_4', '2_5', '2_6',
       '2_7', '3_3'], dtype=object)

In [8]:
df[0][df[0]['unit'].isin(TRAIN_BEARINGS[1])]['unit']

0          1_1
1          1_1
2          1_1
3          1_1
4          1_1
          ... 
2229755    1_2
2229756    1_2
2229757    1_2
2229758    1_2
2229759    1_2
Name: unit, Length: 9405440, dtype: object

In [9]:
# vib_h = df[0]['H_acc'].values
# vib_v = df[0]['V_acc'].values

# Normalize each channel independently
# scaler_h = RobustScaler()
# scaler_v = RobustScaler()
# vib_h_scaled = scaler_h.fit_transform(vib_h.reshape(-1, 1)).flatten()
# vib_v_scaled = scaler_v.fit_transform(vib_v.reshape(-1, 1)).flatten()
# rul = df[0]['rul']

# vib_h_scaled.shape, vib_v_scaled.shape, rul.shape

In [10]:
df[0][df[0]['unit'].isin(TRAIN_BEARINGS[1])]

,H_acc,rul,unit,V_acc
0,0.552,28029,1_1,-0.146
1,0.501,28029,1_1,-0.480
2,0.138,28029,1_1,0.435
3,-0.423,28029,1_1,0.240
4,-0.802,28029,1_1,0.020
...,...,...,...,...
2229755,-2.595,0,1_2,3.622
2229756,-2.869,0,1_2,8.675
2229757,1.457,0,1_2,8.111
2229758,1.591,0,1_2,1.403


In [11]:
class BearingDataset(torch.utils.data.Dataset):
    def __init__(self, df, window_size=512, step=64):
        self.windows = []
        self.ruls = []
        
        vib_h = df['H_acc'].values
        vib_v = df['V_acc'].values
        rul = df['rul'].values
        
        # Normalize per bearing
        vib_h = (vib_h - vib_h.mean()) / (vib_h.std() + 1e-8)
        vib_v = (vib_v - vib_v.mean()) / (vib_v.std() + 1e-8)
        
        # Create sequences
        for i in range(0, len(vib_h) - window_size, step):
            self.windows.append(np.stack([vib_h[i:i+window_size], 
                                        vib_v[i:i+window_size]], axis=1))
            self.ruls.append(rul[i+window_size])
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.windows[idx]), torch.FloatTensor([self.ruls[idx]])
    
dataset = BearingDataset(
    df=df[0][df[0]['unit'].isin(TRAIN_BEARINGS[1])],
)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

In [12]:
class BearingCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv1d(2, 64, kernel_size=50, padding=24),  # (B, 2, 512) -> (B, 64, 512)
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),  # -> (B, 64, 256)
            
            torch.nn.Conv1d(64, 128, kernel_size=25, padding=12),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),  # -> (B, 128, 128)
            
            torch.nn.Conv1d(128, 256, kernel_size=10, padding=5),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2)   # -> (B, 256, 64)
        )
        
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(256*64, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(128, 1)
        )
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # (B, 512, 2) -> (B, 2, 512)
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.regressor(x).squeeze()

model = BearingCNN()
print(model)

BearingCNN(
  (features): Sequential(
    (0): Conv1d(2, 64, kernel_size=(50,), stride=(1,), padding=(24,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(64, 128, kernel_size=(25,), stride=(1,), padding=(12,))
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(128, 256, kernel_size=(10,), stride=(1,), padding=(5,))
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (regressor): Sequential(
    (0): Linear(in_features=16384, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

# Early stopping
# best_loss = float('inf')
# patience = 10
# epochs_no_improve = 0
train_r2_history = []

for epoch in range(3):
    model.train()

    train_loss = 0
    epoch_train_preds = []
    epoch_train_targets = []

    for X_batch, y_batch in tqdm(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        y_pred = model(X_batch).flatten()

        epoch_train_preds.append(y_pred)
        epoch_train_targets.append(y_batch)

        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_preds = torch.cat(epoch_train_preds)
    train_targets = torch.cat(epoch_train_targets)
    train_r2 = calculate_r2(train_preds, train_targets)
    train_r2_history.append(train_r2)
    
    # Validation
    # model.eval()
    # with torch.no_grad():
    #     val_pred = model(dataset.X.to(device)).flatten()
    #     val_loss = criterion(val_pred, dataset.y.to(device))
    
    print(f'Epoch {epoch}: Train Loss {train_loss/len(train_loader):.4f}, Train R² = {train_r2:.3f}')
    
    # Early stopping
    # if val_loss < best_loss:
    #     best_loss = val_loss
    #     epochs_no_improve = 0
    #     torch.save(model.state_dict(), 'best_model.pth')
    # else:
    #     epochs_no_improve += 1
    #     if epochs_no_improve == patience:
    #         print('Early stopping!')
    #         break

  0%|          | 0/4593 [00:00<?, ?it/s]/home/stepan/Private/bearing_rul/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|█████████▉| 4584/4593 [00:41<00:00, 99.65it/s] /home/stepan/Private/bearing_rul/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 4593/4593 [00:42<00:00, 109.17it/s]


Epoch 0: Train Loss 11997063.6777, Train R² = 0.824


  0%|          | 0/4593 [00:00<?, ?it/s]/home/stepan/Private/bearing_rul/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|█████████▉| 4587/4593 [01:41<00:00, 61.56it/s]/home/stepan/Private/bearing_rul/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 4593/4593 [01:41<00:00, 45.25it/s]


Epoch 1: Train Loss 3859313.7764, Train R² = 0.944


  0%|          | 0/4593 [00:00<?, ?it/s]/home/stepan/Private/bearing_rul/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|█████████▉| 4592/4593 [03:47<00:00, 10.39it/s]/home/stepan/Private/bearing_rul/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 4593/4593 [03:47<00:00, 20.19it/s]

Epoch 2: Train Loss 4339864.7245, Train R² = 0.936
